In [1]:
import pandas as pd
import numpy as np
import os  
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score, classification_report


In [9]:
def filepath(f):
    d = os.path.join(os.path.dirname(os.getcwd()), 'processed_data', f)
    return d

In [10]:
#Import training and test datasets
train_data=pd.read_csv(filepath("final_training_set.csv"))
test_data=pd.read_csv(filepath("final_test_set.csv"))

In [130]:
#test_data.columns

In [131]:
# List of columns to exclude from the model
columns_to_exclude = ['Provider', 'BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt','AdmissionDt', 'DischargeDt', 'DOB', 'DOD','diagnosis_descriptor','procedures_descriptor']

train_data = train_data.drop(columns=columns_to_exclude)
train_data = train_data.drop(columns='RenalDiseaseIndicator')
train_data = train_data.drop(columns='DeductableAmtPaid')
test_data = test_data.drop(columns=columns_to_exclude)

KeyError: "['DeductableAmtPaid'] not found in axis"

In [ ]:
#Define variables needed
target = 'PotentialFraud'
features = [col for col in train_data.columns if col != target]


In [ ]:
X_train = train_data[features]
#X_train['DeductibleAmtPaid'].fillna(0, inplace=True)
y_train = train_data[target]

C:\Users\Bryan-PC\AppData\Local\Temp\ipykernel_21072\1831840607.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['DeductibleAmtPaid'].fillna(0, inplace=True)


In [ ]:
# Initialize and train the Isolation Forest model
model = IsolationForest(contamination=0.1, random_state=42)
model.fit(X_train)

IsolationForest(contamination=0.1, random_state=42)

In [ ]:
X_test = test_data[features]
X_test = test_data.drop(columns='DeductableAmtPaid')
test_data['PotentialFraud'] = 0
y_test = test_data[target]

In [ ]:
# Predict on the test set
y_pred = model.predict(X_test)

# The Isolation Forest model assigns -1 to anomalies (fraud) and 1 to normal data hence we convert -1 to 0
y_pred_binary = [1 if label == -1 else 0 for label in y_pred]

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred_binary)
# Calculate the classification report with zero_division=1
report = classification_report(y_test, y_pred_binary, zero_division=1)

In [132]:
print(report)

              precision    recall  f1-score   support

           0       1.00      0.90      0.95    135392
           1       0.00      1.00      0.00         0

    accuracy                           0.90    135392
   macro avg       0.50      0.95      0.47    135392
weighted avg       1.00      0.90      0.95    135392

